# Welcome to a very busy Week 8 folder

## We have lots to do this week!

We'll move at a faster pace than usual, particularly as you're becoming proficient LLM engineers.


In [5]:
pip install modal

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# Just one import to start with!!

import modal

# Setting up the modal tokens

The first time you run this, please uncomment the next line and execute it.  
This is the same as running `modal setup` from the command line. It connects with Modal and installs your tokens.  

## Debugging some common problems on Windows

If this command fails in the next cell, or if any of the modal commands with the `!` fail, please try running them directly on the command line in an activated environment (without the `!`)

A student on Windows mentioned that on Windows, you might also need to run this command from a command prompt in an activated environment afterwards:  
`modal token new`  
(Thank you Ed B. for that!)

Also, a student David S. mentioned the following:  
> In case anyone else using Windows hits this problem: Along with having to run `modal token new` from a command prompt, you have to move the generated token file. It will deploy the token file (.modal.toml) to your Windows profile folder. The virtual environment couldn't see that location (strangely, it couldn't even after I set environment variables for it and rebooted). I moved that token file to the folder I'm operating out of for the lab and it stopped throwing auth errors.

Finally: I've also heard that in some situations, you might need to restart the Kernel of this jupyter notebook after running this. (Kernel menu >> Restart Kernel and Clear Outputs of All Cells).

In [3]:
# Remove the '# ' from the next line and run the cell
!modal setup

+- Error ---------------------------------------------------------------------+
| Could not connect to the Modal server.                                      |
+-----------------------------------------------------------------------------+


In [4]:
!modal token new

+- Error ---------------------------------------------------------------------+
| Could not connect to the Modal server.                                      |
+-----------------------------------------------------------------------------+


In [5]:
!modal token set --token-id ak-jQHflODSwGNvab97cT9lRY --token-secret as-DKP5ZJY5GM3AG1LdZm6x32

Verifying token against https://api.modal.com

+- Error ---------------------------------------------------------------------+
| Could not connect to the Modal server.                                      |
+-----------------------------------------------------------------------------+


In [3]:
!curl -I https://api.modal.com

HTTP/1.1 502 Bad Gateway
Date: Thu, 20 Mar 2025 17:26:26 GMT
Content-Type: text/html
Content-Length: 150
Connection: close
Strict-Transport-Security: max-age=31536000; includeSubDomains



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0   150    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


In [14]:
!ping api.modal.com


Pinging api.modal.com [54.158.199.14] with 32 bytes of data:
Request timed out.
Request timed out.
Request timed out.
Request timed out.

Ping statistics for 54.158.199.14:
    Packets: Sent = 4, Received = 0, Lost = 4 (100% loss),


In [11]:
!modal --version

modal client version: 0.73.116


In [12]:
import os
os.environ["MODAL_TOKEN_ID"] = "ak-jQHflODSwGNvab97cT9lRY"
os.environ["MODAL_TOKEN_SECRET"] = "as-DKP5ZJY5GM3AG1LdZm6x32"

In [15]:
!modal profile list

+- Error ---------------------------------------------------------------------+
| Could not connect to the Modal server.                                      |
+-----------------------------------------------------------------------------+


In [9]:
cat /root/.modal.toml

SyntaxError: invalid syntax (3033802001.py, line 1)

In [10]:
%cd /content/drive/MyDrive/Multi-Agent-System
!ls -lah  # Check if hello.py is there


[WinError 3] The system cannot find the path specified: '/content/drive/MyDrive/Multi-Agent-System'
c:\Users\david.doggala\OneDrive - ascendion\Desktop\Langchain\LLM_Engg\Multi-Agent-System


'ls' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Multi-Agent-System')

In [ ]:
from hello import app, hello

/content/drive/MyDrive/Multi-Agent-System/hello.py:8: DeprecationError: 2025-02-03: Modal will stop implicitly adding local Python modules to the Image ("automounting") in a future update. The following modules need to be explicitly added for future compatibility:
* modal_tokens

e.g.:
image_with_source = my_image.add_local_python_source("modal_tokens")

For more information, see https://modal.com/docs/guide/modal-1-0-migration
  @app.function(image=image)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with app.run():
    reply=hello.local()
print(reply)

Hello from North Charleston, South Carolina, US!!


In [ ]:
# Cell 1: Install, set tokens, and define files

# Write modal_tokens.py (for local token setting)
with open("modal_tokens.py", "w") as f:
    f.write("""
def load_tokens():
    import os
    os.environ["MODAL_TOKEN_ID"] = "ak-jQHflODSwGNvab97cT9lRY"
    os.environ["MODAL_TOKEN_SECRET"] = "as-DKP5ZJY5GM3AG1LdZm6x32"
""")

# Write hello.py (no modal_tokens import)
with open("hello.py", "w") as f:
    f.write("""
import modal
from modal import App, Image

app = modal.App("hello-v2")  # Unique name to avoid old deployment
image = Image.debian_slim().pip_install("requests")

@app.function(image=image)
def hello() -> str:
    import requests
    response = requests.get('https://ipinfo.io/json')
    data = response.json()
    city, region, country = data['city'], data['region'], data['country']
    return f"Hello from {city}, {region}, {country}!!"
""")

# Set tokens locally and import app
from modal_tokens import load_tokens
load_tokens()
from hello import app, hello

# Deploy the app (optional explicit deployment)
with app.run():
    print("App deployed successfully")

App deployed successfully


In [ ]:
# Cell 2: Run remotely
from hello import app, hello

with app.run():
    reply = hello.remote()
    print(reply)

Hello from Ashburn, Virginia, US!!


# Before we move on -

## We need to set your HuggingFace Token as a secret in Modal

1. Go to modal.com, sign in and go to your dashboard
2. Click on Secrets in the nav bar
3. Create new secret, click on Hugging Face, this new secret needs to be called **hf-secret** because that's how we refer to it in the code
4. Fill in your HF_TOKEN where it prompts you

### And now back to business: time to work with Llama

In [ ]:
# Install modal

# Set Modal tokens locally
with open("modal_tokens.py", "w") as f:
    f.write("""
def load_tokens():
    import os
    os.environ["MODAL_TOKEN_ID"] = "ak-jQHflODSwGNvab97cT9lRY"
    os.environ["MODAL_TOKEN_SECRET"] = "as-DKP5ZJY5GM3AG1LdZm6x32"
""")

# Define the app in llama.py with huggingface-secret and a new app name
with open("llama.py", "w") as f:
    f.write("""
import modal
from modal import App, Volume, Image

app = modal.App("llama-v3")  # New unique name
image = Image.debian_slim().pip_install("torch", "transformers", "bitsandbytes", "accelerate")
secrets = [modal.Secret.from_name("huggingface-secret")]
GPU = "T4"
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

@app.function(image=image, secrets=secrets, gpu=GPU, timeout=1800)
def generate(prompt: str) -> str:
    import torch
    from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, set_seed

    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4"
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=quant_config,
        device_map="auto"
    )

    set_seed(42)
    inputs = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
    attention_mask = torch.ones(inputs.shape, device="cuda")
    outputs = model.generate(inputs, attention_mask=attention_mask, max_new_tokens=5, num_return_sequences=1)
    return tokenizer.decode(outputs[0])
""")

# Load tokens and import app
from modal_tokens import load_tokens
load_tokens()

# Clear any old module cache
import sys
if "llama" in sys.modules:
    del sys.modules["llama"]

from llama import app, generate

# Verify the file contents
print("Contents of llama.py:")
!cat llama.py

Contents of llama.py:

import modal
from modal import App, Volume, Image

app = modal.App("llama-v3")  # New unique name
image = Image.debian_slim().pip_install("torch", "transformers", "bitsandbytes", "accelerate")
secrets = [modal.Secret.from_name("huggingface-secret")]
GPU = "T4"
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

@app.function(image=image, secrets=secrets, gpu=GPU, timeout=1800)
def generate(prompt: str) -> str:
    import torch
    from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, set_seed

    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4"
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME, 
        quantization_config=quant_config

/content/drive/MyDrive/Multi-Agent-System/llama.py:11: DeprecationError: 2025-02-03: Modal will stop implicitly adding local Python modules to the Image ("automounting") in a future update. The following modules need to be explicitly added for future compatibility:
* hello
* modal_tokens

e.g.:
image_with_source = my_image.add_local_python_source("hello", "modal_tokens")

For more information, see https://modal.com/docs/guide/modal-1-0-migration
  @app.function(image=image, secrets=secrets, gpu=GPU, timeout=1800)


In [ ]:
# Run remotely with output
from llama import app, generate

with modal.enable_output():  # Show Modal logs
    with app.run():
        result = generate.remote("Life is a mystery, everyone must stand alone, I hear")
        print(result)

Output()

✓ Initialized. View run at https://modal.com/apps/pauldavid551/main/ap-bHZtDu482Elp7MMffNOMvz

Output()

✓ Created objects.
├── 🔨 Created mount PythonPackage:modal_tokens
├── 🔨 Created mount /content/drive/MyDrive/Multi-Agent-System/llama.py
├── 🔨 Created mount PythonPackage:hello
└── 🔨 Created function generate.

Output()

<s> Life is a mystery, everyone must stand alone, I hear the sound of the wind

Stopping app - local entrypoint completed.


✓ App completed. View run at https://modal.com/apps/pauldavid551/main/ap-bHZtDu482Elp7MMffNOMvz

In [ ]:
import modal
from pricer_ephemeral import app, price

In [ ]:
# Set Modal tokens locally
with open("modal_tokens.py", "w") as f:
    f.write("""
def load_tokens():
    import os
    os.environ["MODAL_TOKEN_ID"] = "ak-jQHflODSwGNvab97cT9lRY"
    os.environ["MODAL_TOKEN_SECRET"] = "as-DKP5ZJY5GM3AG1LdZm6x32"
""")

# Define the app in llama.py with huggingface-secret and a new app name
with open("price_ephemeral.py", "w") as f:
    f.write("""import modal
from modal import App, Image

# Setup

app = modal.App("pricer")
image = Image.debian_slim().pip_install("torch", "transformers", "bitsandbytes", "accelerate", "peft")
secrets = [modal.Secret.from_name("huggingface-secret")]

# Constants

GPU = "T4"
BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
PROJECT_NAME = "pricer"
HF_USER = "ed-donner" # your HF name here! Or use mine if you just want to reproduce my results.
RUN_NAME = "2024-09-13_13.04.39"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
REVISION = "e8d637df551603dc86cd7a1598a8f44af4d7ae36"
FINETUNED_MODEL = f"{HF_USER}/{PROJECT_RUN_NAME}"


@app.function(image=image, secrets=secrets, gpu=GPU, timeout=1800)
def price(description: str) -> float:
    import os
    import re
    import torch
    from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, set_seed
    from peft import PeftModel

    QUESTION = "How much does this cost to the nearest dollar?"
    PREFIX = "Price is $"

    prompt = f"{QUESTION}\n{description}\n{PREFIX}"

    # Quant Config
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4"
    )

    # Load model and tokenizer

    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        quantization_config=quant_config,
        device_map="auto"
    )


    set_seed(42)
    inputs = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
    attention_mask = torch.ones(inputs.shape, device="cuda")
    outputs = base_model.generate(inputs, attention_mask=attention_mask, max_new_tokens=5, num_return_sequences=1)
    result = tokenizer.decode(outputs[0])

    contents = result.split("Price is $")[1]
    contents = contents.replace(',','')
    match = re.search(r"[-+]?\d*\.\d+|\d+", contents)
    return float(match.group()) if match else 0

""")

# Load tokens and import app
from modal_tokens import load_tokens
load_tokens()

# Clear any old module cache
import sys
if "pricer_ephemeral" in sys.modules:
    del sys.modules["pricer_ephemeral"]

from pricer_ephemeral import app, price

# Verify the file contents
print("Contents of prices.py:")
!cat pricer_ephemeral.py

Contents of prices.py:
import modal
from modal import App, Image

# Setup

app = modal.App("pricer")
image = Image.debian_slim().pip_install("torch", "transformers", "bitsandbytes", "accelerate", "peft")
secrets = [modal.Secret.from_name("huggingface-secret")]

# Constants

GPU = "T4"
BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
PROJECT_NAME = "pricer"
HF_USER = "ed-donner" # your HF name here! Or use mine if you just want to reproduce my results.
RUN_NAME = "2024-09-13_13.04.39"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
REVISION = "e8d637df551603dc86cd7a1598a8f44af4d7ae36"
FINETUNED_MODEL = f"{HF_USER}/{PROJECT_RUN_NAME}"


@app.function(image=image, secrets=secrets, gpu=GPU, timeout=1800)
def price(description: str) -> float:
    import os
    import re
    import torch
    from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, set_seed
    from peft import PeftModel

    QUESTION = "How much does this cost to the nearest dollar?"
    PREFIX = "P

/content/drive/MyDrive/Multi-Agent-System/pricer_ephemeral.py:22: DeprecationError: 2025-02-03: Modal will stop implicitly adding local Python modules to the Image ("automounting") in a future update. The following modules need to be explicitly added for future compatibility:
* hello
* llama
* modal_tokens

e.g.:
image_with_source = my_image.add_local_python_source("hello", "llama", "modal_tokens")

For more information, see https://modal.com/docs/guide/modal-1-0-migration
  @app.function(image=image, secrets=secrets, gpu=GPU, timeout=1800)


In [ ]:
with modal.enable_output():
    with app.run():
        result=price.remote("Quadcast HyperX condenser mic, connects via usb-c to your computer for crystal clear audio")
        print(result)

Output()

✓ Initialized. View run at https://modal.com/apps/pauldavid551/main/ap-Yi62xERjIbWHUpWLdJM8SS

Output()

✓ Created objects.
├── 🔨 Created mount PythonPackage:modal_tokens
├── 🔨 Created mount /content/drive/MyDrive/Multi-Agent-System/pricer_ephemeral.py
├── 🔨 Created mount PythonPackage:llama
├── 🔨 Created mount PythonPackage:hello
└── 🔨 Created function price.

Output()

Disconnecting from Modal - This will terminate your Modal app in a few seconds.

Stopping app - keyboard interrupt received. Use `modal run --detach` to prevent KeyboardInterrupts from killing apps.


Runner terminated.


✓ App aborted. View run at https://modal.com/apps/pauldavid551/main/ap-Yi62xERjIbWHUpWLdJM8SS

## Transitioning From Ephemeral Apps to Deployed Apps

From a command line, `modal deploy xxx` will deploy your code as a Deployed App

This is how you could package your AI service behind an API to be used in a Production System.

You can also build REST endpoints easily, although we won't cover that as we'll be calling direct from Python.

## Important note for Windows people:

On the next line, I call `modal deploy` from within Jupyter lab; I've heard that on some versions of Windows this gives a strange unicode error because modal prints emojis to the output which can't be displayed. If that happens to you, simply use an Anaconda Prompt window or a Powershell instead, with your environment activated, and type `modal deploy pricer_service` there. Follow the same approach the next time we do `!modal deploy` too.

As an alternative, a few students have mentioned you can run this code within Jupyter Lab if you want to run it from here:
```
# Check the default encoding
print(locale.getpreferredencoding())  # Should print 'UTF-8'

# Ensure UTF-8 encoding
os.environ['PYTHONIOENCODING'] = 'utf-8'
```

In [ ]:
!modal deploy pricer_service

╭─ Modal Deprecation Warning (2025-02-06) ─────────────────────────────────────────────────────────╮
│ Using Python module paths will require using the -m flag in a future version of Modal.           │
│ Use `modal deploy -m pricer_service` instead.                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
⠸ Creating objects...
⠦ Creating objects...
⠏ Creating objects...
└── ⠦ Creating mount /content/drive/MyDrive/Multi-Agent-System/pricer_service.py: Finalizing index 
⠋ Creating objects...
├── 🔨 Created mount /content/drive/MyDrive/Multi-Agent-System/pricer_service.py
└── 🔨 Created function price.
✓ Created objects.
├── 🔨 Created mount /content/drive/MyDrive/Multi-Agent-System/pricer_service.py
└── 🔨 Created function price.
✓ App deployed in 1.026s! 🎉

View Deployment: https://modal.com/apps/pauldavid551/main/deployed/pricer-service


In [ ]:
pricer = modal.Function.lookup("pricer-service", "price")

/usr/lib/python3.11/asyncio/events.py:84: DeprecationError: 2025-01-27: `modal.Function.lookup` is deprecated and will be removed in a future release. It can be replaced with `modal.Function.from_name`.

See https://modal.com/docs/guide/modal-1-0-migration for more information.
  self._context.run(self._callback, *self._args)


In [ ]:
pricer.remote("Quadcast HyperX condenser mic, connects via usb-c to your computer for crystal clear audio")

199.9

In [ ]:
# You can also run "modal deploy pricer_service2" at the command line in an activated environment
!modal deploy pricer_service2

╭─ Modal Deprecation Warning (2025-02-06) ─────────────────────────────────────────────────────────╮
│ Using Python module paths will require using the -m flag in a future version of Modal.           │
│ Use `modal deploy -m pricer_service2` instead.                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─ Modal Deprecation Warning (2025-01-15) ─────────────────────────────────────────────────────────╮
│ The `@modal.build` decorator is deprecated and will be removed in a future release.              │
│                                                                                                  │
│ We now recommend storing large assets (such as model weights) using a `modal.Volume` instead of  │
│ writing them directly into the `modal.Image` filesystem. For other use cases we recommend using  │
│ `Image.run_function` instead.                                                            

In [ ]:
Pricer = modal.Cls.lookup("pricer-service", "Pricer")
pricer = Pricer()
reply = pricer.price.remote("Quadcast HyperX condenser mic, connects via usb-c to your computer for crystal clear audio")
print(reply)

/usr/lib/python3.11/asyncio/events.py:84: DeprecationError: 2025-01-27: `modal.Cls.lookup` is deprecated and will be removed in a future release. It can be replaced with `modal.Cls.from_name`.

See https://modal.com/docs/guide/modal-1-0-migration for more information.
  self._context.run(self._callback, *self._args)


199.9


# Optional: Keeping Modal warm

## A way to improve the speed of the Modal pricer service

A student mentioned to me that he was concerned by how slow Modal seems to be. The reason is that Modal puts our service to sleep if we don't use it, and then it takes 2.5 minutes to spin back up.

I've added a utility called `keep_warm.py` that will keep our Modal warm by pinging it every 30 seconds.

To use the utliity, bring up a new Terminal (Mac) or Anaconda prompt (Windows), ensure the environment is activated with `conda activate llms`

Then run: `python keep_warm.py` from within the week8 drectory.

Remember to press ctrl+C or exit the window when you no longer need Modal running.


## And now introducing our Agent class

In [ ]:
from agents.specialist_agent import SpecialistAgent

In [ ]:
agent = SpecialistAgent()
agent.price("iPad Pro 2nd generation")

/usr/lib/python3.11/asyncio/events.py:84: DeprecationError: 2025-01-27: `modal.Cls.lookup` is deprecated and will be removed in a future release. It can be replaced with `modal.Cls.from_name`.

See https://modal.com/docs/guide/modal-1-0-migration for more information.
  self._context.run(self._callback, *self._args)


1299.0